<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/beta/af/examples/partial_hallucination_rewire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rewire

In [ ]:
#@title install
%%bash
if [ ! -d params ]; then
  pip -q install git+https://github.com/sokrypton/af_backprop.git
  pip -q install git+https://github.com/sokrypton/ColabDesign.git@beta
  mkdir params
  curl -fsSL https://storage.googleapis.com/alphafold/alphafold_params_2021-07-14.tar | tar x -C params
fi

In [ ]:
#@title import libraries
import os
from af import mk_design_model, clear_mem
from IPython.display import HTML
from google.colab import files
import numpy as np

#########################
def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  else:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"

# partial hallucination

In [ ]:
# setup the model
clear_mem()
model = mk_design_model(protocol="partial",
                        use_templates=False) # set True, to constrain positions using template input

In [ ]:
# define positions we want to restrain/constrain
old_pos = "4-9,13-26,30-33,37-50,54-59,63-76,80-84,112-125"

model.prep_inputs(get_pdb("2MR5"), chain="A",
                  pos=old_pos,               # define positions
                  length=100,                # define if the desire length is different from input PDB
                  fix_seq=False,             # set True, to constrain the sequence
                  sidechains=False)          # set True, to constrain the sidechains

In [ ]:
# get length for each segment
seg_len = [b-a+1 for a,b in [[int(x) for x in r.split("-")] for r in old_pos.split(",")]]

# define order of segments
new_order = [ 1,2,3,0,7,4,5,6]

# define loop lengths between segments (including "loop" before/after)
loop_len =  [0,3,3,3,4,3,3,4,0]

# set positions we want to restrain/constrain within hallucinated protein 
l,new_pos = loop_len[0],[]
for n,i in enumerate(np.argsort(new_order)):
  new_pos.append(l + np.arange(seg_len[i]))
  l += seg_len[i] + loop_len[n+1]
new_pos = np.concatenate([new_pos[i] for i in new_order])

In [ ]:
# restart optimization
model.restart()
# set positions (if different from PDB)
model.opt["pos"] = new_pos
# balance weights dgram_cce=restraint weight, con=hallucination weight
model.opt["weights"].update({"dgram_cce":1.0,"con":0.1})
model.design_3stage(200,100,10)

In [ ]:
HTML(model.animate(dpi=100))

In [ ]:
model.save_pdb(f"{model.protocol}.pdb")
model.plot_pdb()